In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# ✅ 브라우저 초기화 및 로그인
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수
def login_to_console(driver):
    try:
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        pyperclip.copy("dksrufp0607@naver.com")
        id_input.send_keys(Keys.CONTROL, 'v')

        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        pyperclip.copy("ghkfkd159!")
        pw_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(1)
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
        )
        login_button.click()
        print("✅ 로그인 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동
def navigate_to_page(driver):
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    print("✅ 페이지 로드 완료!")

# ✅ "확인" 버튼 클릭
def confirm_click(driver):
    try:
        print("🔍 '확인' 버튼 탐색 중...")
        confirm_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
        )
        confirm_button.click()
        print("✅ 확인 버튼 클릭 완료!")
    except TimeoutException:
        print("⚠️ '확인' 버튼을 찾지 못했습니다. 페이지가 제대로 로드되었는지 확인하세요.")
    except Exception as e:
        print(f"⚠️ 확인 버튼 클릭 실패: {e}")

# ✅ CSV 데이터 로드 함수 (검색용)
def get_csv_value(row, col):
    csv_path = "C:/엑셀/Ranking_1.csv"
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            return df.iloc[row, col]
        else:
            print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
            return None
    except FileNotFoundError:
        print(f"⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# ✅ 검색 실행 함수
def execute_tasks(driver, row, col):
    try:
        b2_value = get_csv_value(row, col)
        if not b2_value:
            print("⚠️ CSV에서 값을 가져오지 못했습니다. 작업을 중단합니다.")
            return

        table_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]'))
        )
        table_dropdown.click()
        print("✅ 테이블 드롭다운 클릭 완료!")

        base_data_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@role="option"][span[text()="BASE_DATA"]]'))
        )
        base_data_option.click()
        print("✅ BASE_DATA 선택 완료!")

        detail_search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]'))
        )
        time.sleep(3)
        try:
            detail_search_button.click()
        except ElementClickInterceptedException:
            print("⚠️ '상세 검색' 버튼 클릭 차단됨. JavaScript로 재시도...")
            driver.execute_script("arguments[0].click();", detail_search_button)
        print("✅ 상세 검색 클릭 완료!")

        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "defaultSearchValue"))
        )
        search_input.clear()
        search_input.send_keys(b2_value)
        print(f"✅ B2 값 입력 완료: {b2_value}")

        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]'))
        )
        search_button.click()
        print("✅ 검색 버튼 클릭 완료!")

        uuid_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]'))
        )
        uuid_element.click()
        print("✅ UUID 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 검색 작업 중 오류 발생: {e}")

# ✅ 엑셀에서 데이터 로드 함수
def load_excel_data(filename):
    try:
        df = pd.read_excel(filename)
        data = df[["Column Name", "Data Type", "Value"]].values.tolist()
        print(f"✅ 엑셀 데이터 로드 완료! 경로: {os.path.abspath(filename)}")
        return data
    except FileNotFoundError:
        print(f"⚠️ 엑셀 파일을 찾을 수 없습니다: {filename}")
        return []
    except Exception as e:
        print(f"⚠️ 엑셀 파일 로드 중 오류 발생: {e}")
        return []

# ✅ 데이터 붙여넣기 함수 (컬럼명만 비교하고 값만 변경)
def paste_data(driver, data):
    try:
        edit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]'))
        )
        edit_button.click()
        print("✅ 수정 버튼 클릭 완료!")
        time.sleep(1)

        # 기본 콘텐츠와 iframe 모두 확인
        driver.switch_to.default_content()
        context_found = False

        # 기본 콘텐츠 확인
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
            )
            print("🔍 기본 콘텐츠에서 요소 발견!")
            context_found = True
        except TimeoutException:
            print("⚠️ 기본 콘텐츠에서 요소 없음")

        # iframe 확인
        if not context_found:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            if iframes:
                print(f"🔍 iframe 감지됨 ({len(iframes)}개):")
                for i, frame in enumerate(iframes):
                    driver.switch_to.frame(frame)
                    print(f"🔍 iframe {i} HTML: {driver.page_source[:500]}")
                    try:
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
                        )
                        print(f"🔍 iframe {i}에서 요소 발견!")
                        context_found = True
                        break
                    except TimeoutException:
                        print(f"⚠️ iframe {i}에서 요소 없음")
                        driver.switch_to.default_content()
            if not context_found:
                print("⚠️ 모든 컨텍스트에서 요소를 찾지 못함")
                return

        for col_name, _, new_value in data:  # data_type은 사용하지 않음
            print(f"🔍 처리 중: {col_name}")
            try:
                elements = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder-edit-modal_1rntr_47"))
                )
                print(f"🔍 elements 개수: {len(elements)}")
                found = False
                for i, element in enumerate(elements):
                    try:
                        print(f"🔹 element {i} 탐색 중...")
                        col_input = element.find_element(By.CSS_SELECTOR, 'input[type="text"][name$=".columnName"]')
                        current_col_name = col_input.get_attribute("value")
                        print(f"🔹 element {i}: {current_col_name}")
                        if current_col_name == col_name:
                            found = True
                            # 데이터 타입 변경 생략
                            try:
                                value_field = element.find_element(By.CSS_SELECTOR, 'textarea[name$=".dataValue"]')
                                value_field.clear()
                                value_field.send_keys(str(new_value))
                                print(f"✅ {col_name} 값 입력 완료: {new_value}")
                            except NoSuchElementException:
                                print(f"⚠️ {col_name} 값 필드가 없음")
                            break
                    except StaleElementReferenceException:
                        print(f"⚠️ element {i}: Stale 발생, 재탐색...")
                        continue
                    except NoSuchElementException:
                        print(f"⚠️ element {i}: input 없음")
                        continue
                if not found:
                    print(f"⚠️ {col_name} 찾을 수 없음")
            except TimeoutException:
                print(f"⚠️ {col_name} 요소 탐색 시간 초과")

        driver.switch_to.default_content()
        save_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "저장")]'))
        )
        save_button.click()
        print("✅ 저장 완료!")
    except Exception as e:
        print(f"⚠️ 데이터 붙여넣기 실패: {e}")

# ✅ 메인 실행
if __name__ == "__main__":
    excel_filename = "C:\\JJG\\Start\\sjdj\\scraped_data2.xlsx"
    print(f"🔍 현재 작업 디렉토리: {os.getcwd()}")
    if not os.path.exists(excel_filename):
        print(f"⚠️ 파일이 존재하지 않습니다: {excel_filename}")
        exit()

    data = load_excel_data(excel_filename)
    if not data:
        print("⚠️ 데이터 로드 실패로 인해 프로그램을 종료합니다.")
        exit()

    driver1 = setup_browser(0)
    try:
        login_to_console(driver1)
        navigate_to_page(driver1)
        confirm_click(driver1)
        execute_tasks(driver1, 0, 5)
        paste_data(driver1, data)
    except KeyboardInterrupt:
        print("⚠️ 사용자가 프로그램을 중단했습니다.")
    finally:
        driver1.quit()
        print("✅ 브라우저가 종료되었습니다.")

🔍 현재 작업 디렉토리: c:\JJG\Start\sjdj
✅ 엑셀 데이터 로드 완료! 경로: C:\JJG\Start\sjdj\scraped_data2.xlsx
✅ 로그인 완료!
✅ 페이지 로드 완료!
🔍 '확인' 버튼 탐색 중...
✅ 확인 버튼 클릭 완료!
🔹 0행 3열 값 (출력용): 슈바니
✅ 테이블 드롭다운 클릭 완료!
✅ BASE_DATA 선택 완료!
✅ 상세 검색 클릭 완료!
✅ B2 값 입력 완료: 0831d8a0-c410-11ef-ac3c-67eaa6cff280
✅ 검색 버튼 클릭 완료!
✅ UUID 클릭 완료!
✅ 수정 버튼 클릭 완료!
🔍 기본 콘텐츠에서 요소 발견!
🔍 처리 중: event8_Receive
🔍 elements 개수: 226
🔹 element 0 탐색 중...
🔹 element 0: event8_Receive
✅ event8_Receive 값 입력 완료: 60
🔍 처리 중: purchaseCurMileage
🔍 elements 개수: 226
🔹 element 0 탐색 중...
🔹 element 0: event8_Receive
🔹 element 1 탐색 중...
🔹 element 1: purchaseCurMileage
⚠️ purchaseCurMileage 값 필드가 없음
🔍 처리 중: hpGrowth2_1
🔍 elements 개수: 226
🔹 element 0 탐색 중...
🔹 element 0: event8_Receive
🔹 element 1 탐색 중...
🔹 element 1: purchaseCurMileage
🔹 element 2 탐색 중...
🔹 element 2: hpGrowth2_1
✅ hpGrowth2_1 값 입력 완료: 308235
🔍 처리 중: mission_Weekly_Check
🔍 elements 개수: 226
🔹 element 0 탐색 중...
🔹 element 0: event8_Receive
🔹 element 1 탐색 중...
🔹 element 1: purchaseCurMileage
🔹 element 2